### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [2]:
# !pip install transformers datasets accelerate deepspeed evaluate sequencepiece
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
import evaluate
from tqdm import tqdm
import numpy as np

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Load data and model

In [3]:
qqp = datasets.load_dataset('SetFit/qqp')
clear_output(wait=True)
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [5]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [12]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

In [7]:
qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [8]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [9]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [10]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
    predicted = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        token_type_ids=batch['token_type_ids']
    )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [5]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.softmax(dim=-1).argmax(-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False, collate_fn=transformers.default_data_collator)
model = model.to(device)

accuracy = 0
with torch.no_grad():
     for batch in tqdm(val_loader):
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        accuracy += compute_metrics((predicted.logits, batch["labels"].to(device)))["accuracy"]

accuracy /= len(val_loader)

100%|██████████| 632/632 [04:37<00:00,  2.27it/s]


In [13]:
assert 0.9 < accuracy < 0.91
accuracy

0.908348238855256

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [15]:
qqp.cleanup_cache_files()

{'train': 6, 'test': 0, 'validation': 0}

In [16]:
''' Select the model and tokenize dataset'''

model_name = "microsoft/deberta-v3-base"
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
clear_output(wait=True)

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 40/41 [00:02<00:00, 17.46ba/s]


In [19]:
''' Compute model accyracy without finetuning '''
val_loader = torch.utils.data.DataLoader(qqp_preprocessed['validation'], batch_size=64, shuffle=False, num_workers=4, collate_fn=transformers.default_data_collator)

model = model.to(device)
accuracy = 0
with torch.no_grad():
     for batch in tqdm(val_loader):
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device),
        )
        accuracy += compute_metrics((predicted.logits, batch["labels"].to(device)))["accuracy"]
accuracy /= len(val_loader)

100%|██████████| 632/632 [05:30<00:00,  1.91it/s]


In [20]:
print(f"{model_name} accuracy without finetune: {accuracy}")

microsoft/deberta-v3-base accuracy without finetune: 0.6317494066455697


In [8]:
''' Train model '''
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="trainer", 
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    warmup_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=1,
    overwrite_output_dir=True,
    logging_steps=1500,
    logging_dir="trainer",
    dataloader_num_workers=8,
    save_strategy="epoch", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=qqp_preprocessed['train'],
    eval_dataset=qqp_preprocessed['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: label_text, idx, text1, text2. If label_text, idx, text1, text2 are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 363846
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 11371


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fas

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss,Accuracy
1500,0.359900,0.273709,0.880905
3000,0.272900,0.250407,0.892580
4500,0.260200,0.243678,0.894534
6000,0.243100,0.232500,0.905837
7500,0.229400,0.226109,0.905788
9000,0.226400,0.223124,0.909374
10500,0.216000,0.214968,0.910611


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fas

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saving model checkpoint to trainer/checkpoint-11371
Configuration saved in trainer/checkpoint-11371/config.json
Model weights saved in trainer/checkpoint-11371/pytorch_model.bin
tokenizer config file saved in trainer/checkpoint-11371/tokenizer_config.json
Special tokens file saved in trainer/checkpoint-11371/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=11371, training_loss=0.25494093175677046, metrics={'train_runtime': 7051.3357, 'train_samples_per_second': 51.6, 'train_steps_per_second': 1.613, 'total_flos': 2.393340547233485e+16, 'train_loss': 0.25494093175677046, 'epoch': 1.0})

In [21]:
''' Compute model accyracy without finetuning '''
val_loader = torch.utils.data.DataLoader(qqp_preprocessed['validation'], batch_size=32, shuffle=False, num_workers=4, collate_fn=transformers.default_data_collator)

model = model.to(device)
accuracy = 0
with torch.no_grad():
     for batch in tqdm(val_loader):
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device),
        )
        accuracy += compute_metrics((predicted.logits, batch["labels"].to(device)))["accuracy"]
accuracy /= len(val_loader)

  0%|                                                                                                                                          | 0/1264 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1263/1264 [03:37<00:00,  5.79it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1264/1264 [03:37<00:00,  5.80it/s]


In [22]:
print(f"{model_name} accuracy after finetune: {accuracy}")

microsoft/deberta-v3-base accuracy after finetune: 0.910375226039783


### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("trainer/checkpoint-11371").to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

In [8]:
full_train_texts = [p["text1"] for p in qqp["train"]]
full_train_texts += [p["text2"] for p in qqp["train"]]
full_train_texts = list(set(full_train_texts))

In [9]:
class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, question, texts):
        self.question = question
        self.texts = texts
    def __getitem__(self, idx):
        return tokenizer(self.question, self.texts[idx], padding='max_length', max_length=MAX_LENGTH, truncation=True)
    def __len__(self):
        return len(self.texts)

In [21]:
def find_duplicates(model, question, full_train_texts, top_k):
    ds = QuestionDataset(question, full_train_texts)
    loader = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=8)
    duplicate_scores = []
    with torch.no_grad():
         for batch in tqdm(loader):
            predicted = model(
                input_ids=batch['input_ids'].to(device),
                attention_mask=batch['attention_mask'].to(device),
                token_type_ids=batch['token_type_ids'].to(device),
            )
            duplicate_scores.append(predicted.logits.softmax(dim=-1)[:, 1])
    duplicate_scores = torch.cat(duplicate_scores).detach().cpu()
    top_k_ids = duplicate_scores.argsort(descending=True)[:top_k]
    top_k_scores = duplicate_scores[top_k_ids].numpy()
    top_k_texts = [full_train_texts[i] for i in top_k_ids]
    print(f"TOP_{top_k} potential duplicates in the training set:")
    for ind in top_k_ids:
        print(f"   SCORE: {duplicate_scores[ind]:.3f} TEXT: {full_train_texts[ind]}")

In [22]:
find_duplicates(model, "What can one do after MBBS?", full_train_texts, top_k=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15434/15434 [38:18<00:00,  6.71it/s]


TOP_5 potential duplicates in the training set:
   SCORE: 0.993 TEXT: What have to Do after MBBS?
   SCORE: 0.992 TEXT: What are the career options after mbbs?
   SCORE: 0.987 TEXT: What can I do after bba course?
   SCORE: 0.983 TEXT: What are the job opportunities after MBBS?
   SCORE: 0.977 TEXT: What should I do after completing my bsc?


In [24]:
find_duplicates(model.to(device), "What can cause stool to come out as little balls?", full_train_texts, top_k=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15434/15434 [38:19<00:00,  6.71it/s]


TOP_5 potential duplicates in the training set:
   SCORE: 0.985 TEXT: What can cause stool to come out as little balls?
   SCORE: 0.328 TEXT: Is dark matter matter and dark energy energy? Since matter can convert to energy, are both dark matter and dark energy considered a form of energy?
   SCORE: 0.081 TEXT: Do raccoons wash their food? Why do raccoons wash their food?
   SCORE: 0.067 TEXT: If Aliens are not found yet, how do people depicting them in such many shapes? Why do they always depicting aliens in a weird shapes?
   SCORE: 0.063 TEXT: Is it possible to make a ghost appear at school in front of you? How can you make a ghost appear in front of you during class or in school?


In [25]:
find_duplicates(model.to(device), "What will be Hilary Clinton's policy towards India if she become President?", full_train_texts, top_k=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15434/15434 [38:22<00:00,  6.70it/s]


TOP_5 potential duplicates in the training set:
   SCORE: 0.997 TEXT: If Hillary Clinton wins this election, what will be her policy for India?
   SCORE: 0.996 TEXT: What will be Hillary Clinton's India policy if she wins the election?
   SCORE: 0.996 TEXT: What will be the Hillary Clinton's India policy if she become the president of USA?
   SCORE: 0.996 TEXT: What will be Hillary Clinton foreign policy with respect to India once she becomes president?
   SCORE: 0.996 TEXT: How would Hillary Clinton keep USA's relationship with India if she becomes president?


In [26]:
find_duplicates(model.to(device), "How to pass an NLP_course and not burn your GPU?", full_train_texts, top_k=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15434/15434 [38:21<00:00,  6.71it/s]


TOP_5 potential duplicates in the training set:
   SCORE: 0.048 TEXT: Is dark matter matter and dark energy energy? Since matter can convert to energy, are both dark matter and dark energy considered a form of energy?
   SCORE: 0.014 TEXT: How can I improve my english within 6 months? Can I improve my english from books?
   SCORE: 0.009 TEXT: Is it possible to make a ghost appear at school in front of you? How can you make a ghost appear in front of you during class or in school?
   SCORE: 0.009 TEXT: If universe expands and vacuum energy is created with it (with no limit),is there infinite potential energy/infinite vacuum energy that can be created?
   SCORE: 0.008 TEXT: Are Chinese characters inefficient? Quora Top Stories from Your Feed Your Quora Digest Are Chinese characters inefficient


In [27]:
find_duplicates(model.to(device), "Is Russia a country of militarism?", full_train_texts, top_k=5)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15434/15434 [38:21<00:00,  6.71it/s]


TOP_5 potential duplicates in the training set:
   SCORE: 0.764 TEXT: Is dark matter matter and dark energy energy? Since matter can convert to energy, are both dark matter and dark energy considered a form of energy?
   SCORE: 0.715 TEXT: Is doing global bba from SP JAIN worth the cost? Is it good? Does bba from SP JAIN has any advantage?
   SCORE: 0.680 TEXT: Can anyone help me to understand BACnet Web Services? Or can anyone help me get in touch with someone who can explain me BACnet Web Services?
   SCORE: 0.557 TEXT: Why do Police is not accepting bribe online? Why they want bribe in cash? Our PM want us to be Cashless! Then why Police want bribe in cash?
   SCORE: 0.435 TEXT: Are Wade Wilson and Slade Wilson connected? In other words are Deadpool and Deathstroke connected?


__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.